In [4]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

from model import *

In [ ]:
model = Model(input_vocabulary_size=test_set.input_vocabulary_size,
              target_vocabulary_size=test_set.target_vocabulary_size,
              num_cnn_channels=test_set.image_channels,
              input_padding_idx=test_set.input_vocabulary.pad_idx,
              target_pad_idx=test_set.target_vocabulary.pad_idx,
              target_eos_idx=test_set.target_vocabulary.eos_idx,
              **flags)